<a href="https://colab.research.google.com/github/alexfazio/toolhouse-cookbook/blob/main/th_tool_ex_code_interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Dependencies

# Install the toolhouse library
%pip install toolhouse openai

print("---")
!pip show toolhouse openai

# Import necessary libraries
from toolhouse import Toolhouse
import os

In [ ]:
# @title Enter API Keys

from getpass import getpass

# Set up the Toolhouse API key
toolhouse_api_key = getpass("Enter your Toolhouse API key: ")
os.environ["TOOLHOUSE_API_KEY"] = toolhouse_api_key

# Set up the Toolhouse API key
openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key


In [ ]:
# @title Check Available Tools

# # Create a Toolhouse instance
provider = "openai" # @param ["openai","anthropic"]
# t = Toolhouse(provider)  # You can change the provider if needed

# Create a Toolhouse instance
t = Toolhouse(provider=provider)  # You can change the provider if needed

# Get the list of available tools
available_tools = t.get_tools()

# Print the list of available tools
if not available_tools:
    print("No tools are currently installed.")
else:
    print("Available tools:")
    for tool in available_tools:
        print(f"- {tool.get('name')}")

print(available_tools)

In [ ]:
# @title Kickoff

from typing import List
# 👋 Make sure you've also installed the OpenAI SDK through: pip install openai
from openai import OpenAI
from toolhouse import Toolhouse

# Let's set our API Keys.
# Please remember to use a safer system to store your API KEYS
# after finishing the quick start.

# Define the OpenAI model we want to use
MODEL = 'gpt-4o'

assert 'code_interpreter' not in available_tools[0], f"'code_interpreter' is not in available_tools.\n\n[available_tools]: {available_tools}"

# Retrieve the API keys from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
toolhouse_api_key = os.getenv("TOOLHOUSE_API_KEY")

# Use the retrieved API keys to initialize your clients
client = OpenAI(api_key=openai_api_key)
th = Toolhouse(access_token=toolhouse_api_key)

th.set_metadata("timezone", -7)

messages: List[dict] = [{
    "role": "user",
    "content":
        "Generate FizzBuzz code."
        "Execute it to show me the results up to 10."
}]

response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  # Passes Code Execution as a tool
  tools=th.get_tools()
)

# Runs the Code Execution tool, gets the result,
# and appends it to the context
messages += th.run_tools(response)

response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  tools=th.get_tools()
)
# Prints the response with the answer
print(response.choices[0].message.content)